# Diving into French Presidential Discourses
## Marine Le Pen (MLP)

In [ ]:
!pip install minet
!pip install pdfminer.six
!pip install unidecode
!python -m spacy download fr_core_news_sm

In [1]:
import pandas as pd
from pdfminer.high_level import extract_text
from gensim.test.utils import datapath
from gensim import utils
import spacy
from collections import Counter
import unidecode
import gensim.models

In [2]:
## create a .csv of MLP's Twitter activity since 27 Jan 2022
!minet tw scrape tweets "(from:MLP_officiel) until:2022-03-27 since:2022-01-27" > tweets_MLP.csv

## convert the .csv file in a data frame using pandas
df_tw_MLP = pd.read_csv("./tweets_MLP.csv")

## create a list of tweets selecting the 'text' column of the data frame
list_tw_MLP = df_tw_MLP['text'].values.tolist()
len(list_tw_MLP)

Searching for "(from:MLP_officiel) until:2022-03-27 since:2022-01-27"


1035

In [3]:
## retrieve MLP's affiliates Twitter activity
!minet tw scrape tweets "(from:de_beaujeu OR from:JulienOdoul OR from:sebchenu OR from:SteeveBriois OR from:ljacobelli OR from:jsanchez_rn OR from:jllacapelle OR from:david_rachline OR from:franckallisio OR from:ThierryMARIANI OR from:BallardPhilippe OR from:louis_aliot OR from:wdesaintjust OR from:BrunoBilde) until:2022-03-27 since:2022-01-27" > tw_MLP_aff_all.csv

## convert MLP's affiliates' tweets in a list
df_tw_MLP_aff_all = pd.read_csv("tw_MLP_aff_all.csv")
list_tw_MLP_aff_all = df_tw_MLP_aff_all['text'].values.tolist()
print(list_tw_MLP_aff_all[0])
print(len(list_tw_MLP_aff_all))

Searching for "(from:de_beaujeu OR from:JulienOdoul OR from:sebchenu OR from:SteeveBriois OR from:ljacobelli OR from:jsanchez_rn OR from:jllacapelle OR from:david_rachline OR from:franckallisio OR from:ThierryMARIANI OR from:BallardPhilippe OR from:louis_aliot OR from:wdesaintjust OR from:BrunoBilde) until:2022-03-27 since:2022-01-27"
Le rôle d’un homme d’Etat est de penser d’abord à son pays et de refuser de se laisser entraîner dans des aventures incertaines par des pressions internationales ou des campagnes médiatiques .
Et #Orban montre dans cette crise qu’il est vraiment un homme d’Etat. « Le_Figaro: «La Hongrie veut rester en dehors de cette guerre et n'autorisera pas le transfert d'armes vers l'Ukraine.» https://www.lefigaro.fr/flash-actu/aux-critiques-de-zelensky-orban-oppose-les-interets-de-la-hongrie-20220325?utm_medium=Social&utm_campaign=echobox&utm_source=Twitter&origine=VWT16001#Echobox=1648222324-1 — https://twitter.com/Le_Figaro/status/1507379798871625728 »
3271


In [4]:
## merge MLP and his affiliates' lists
list_tw_MLP_all = list_tw_MLP + list_tw_MLP_aff_all
print(len(list_tw_MLP_all))

4306


In [5]:
## i retrieve a string from the pdf of MLP's manifesto using extract_text of the pdfminer package
## the cleaning process is specific for this manifesto and it depends on the output of extract_text
manif_MLP = extract_text('/Users/simonemariaparazzoli/Documents/Università/Sciences Po/Diving into public digital spaces/research/manifesto_lepen.pdf')
manif_clean_MLP = manif_MLP.replace('-\n','')
manif_clean_MLP = manif_clean_MLP.replace('\n\n',' ')
manif_clean_MLP = manif_clean_MLP.replace('\n','')
manif_clean_MLP = manif_clean_MLP.replace('\uf0e8\u2009','---')
manif_clean_MLP = manif_clean_MLP.replace('\uf0e8\xa0','---')
manif_clean_MLP = manif_clean_MLP.replace('\x0c',' ')
manif_clean_MLP = manif_clean_MLP.replace('\xa0','')
#print(repr(manif_clean_MLP))

## convert the string of the manifesto into a list
list_manif_MLP = manif_clean_MLP.split("---")
list_manif_MLP = [s for s in list_manif_MLP if len(s)>30]
len(list_manif_MLP)
#print(list_manif_MLP)

66

In [12]:
## merge the two lists of tweets and of the manifesto 
list_MLP = list_tw_MLP_all + list_tw_MLP_all + list_manif_MLP
len(list_MLP)

## load a spacy model to retrieve stop words
nlp = spacy.load("fr_core_news_sm")
stop_words_fr = nlp.Defaults.stop_words

## clean the list of tweets and manifesto to get rid of useless words and make the list content homogeneous
list_MLP_clean = []
for i in list_MLP:
    doc = nlp(i)
    tokens = [unidecode.unidecode(token.text).lower() for token in doc 
              if (token.text not in stop_words_fr and
                  len(token.text)>2 and
                  token.like_url == False )]
    tokens_joined = ' '.join(tokens)
    list_MLP_clean.append(tokens_joined)
    
## test the output of the cleaning process
print(list_MLP[205])
print("---")
print(list_MLP_clean[205])

Je réserverai les prestations de solidarité aux Français, et aux étrangers qui peuvent justifier de 5 ans de travail équivalent temps plein avant d’être amenés à quitter notre pays s’ils sont au chômage depuis plus d’un an.  

👉🏻 https://www.mlafrance.fr https://twitter.com/MLP_officiel/status/1504815315535945730/photo/1
---
reserverai prestations solidarite francais etrangers justifier ans travail equivalent temps plein amenes quitter pays chomage  




In [47]:
## prepare the corpus as a class
class MyCorpus_MLP:

    def __iter__(self):
        for i in list_MLP_clean:
            yield utils.simple_preprocess(i,min_len=3)
            
## train the word embeddings model_MLP
sentences = MyCorpus_MLP()
model_MLP = gensim.models.Word2Vec(sentences=sentences, min_count=10, vector_size=300, epochs=100)

In [48]:
## transform the corpus list (that is made of tweets and sentences from the manifesto)
## in a list containing all the words of the corpus as elements of the list
words_MLP = []

for i in list_MLP_clean:
    i_split = i.split(' ') #transform the i document into a list (split at blank space)
    words_MLP.extend(i_split)

## clean the list of tokens
words_MLP_clean = [x for x in words_MLP 
                   if x not in stop_words_fr
                   if x != "\n\n"
                   if len(x)>1]

## find the 30 most common words using Counter
words_freq_MLP = Counter(words_MLP_clean)
common_words_MLP = words_freq_MLP.most_common(30)
print(common_words_MLP)

[('francais', 1486), ('marinepresidente', 1388), ('@mlp_officiel', 1350), ('france', 1245), ('macron', 1164), ('mlafrance', 614), ('@cnews', 564), ('pouvoir', 549), ('faire', 536), ('faut', 532), ('pays', 496), ('marine', 483), ('ans', 438), ('emmanuel', 418), ('ete', 412), ('politique', 375), ('pen', 375), ('@vpecresse', 374), ('contre', 371), ('@bfmtv', 366), ('ukraine', 362), ('achat', 341), ('russie', 330), ('zemmour', 322), ('presidentielle', 314), ("aujourd'hui", 312), ('bien', 304), ('immigration', 300), ('projet', 296), ('campagne', 278)]


In [49]:
## first attempt with the most_similar function on our corpus using our model_MLP
result = model_MLP.wv.most_similar(positive=['travail'], topn=30)
print(result)

[('meilleure', 0.3576391041278839), ('communes', 0.3172711133956909), ('ceux', 0.3118799030780792), ('souhaitent', 0.31172460317611694), ('tot', 0.29881641268730164), ('fassent', 0.29840052127838135), ('entres', 0.2960669994354248), ('reserver', 0.2939426898956299), ('remettre', 0.29287901520729065), ('aides', 0.29284030199050903), ('annuites', 0.2891264259815216), ('travaille', 0.28369274735450745), ('stephane_ravier', 0.2827950716018677), ('apprentissage', 0.2816009819507599), ('patrimoine', 0.27423107624053955), ('confiance', 0.26825299859046936), ('nos', 0.26811420917510986), ('equivalent', 0.26754337549209595), ('implication', 0.2652222216129303), ('engagement', 0.26294928789138794), ('bien', 0.26139208674430847), ('impot', 0.25969138741493225), ('desormais', 0.2586952745914459), ('regardez', 0.253057599067688), ('determination', 0.24815590679645538), ('ordre', 0.24394625425338745), ('rentrer', 0.2402544915676117), ('revenu', 0.23464198410511017), ('entier', 0.2326968014240265), (

In [50]:
print(len(list_tw_MLP_all),',',len(list_MLP_clean))

4306 , 8678


In [51]:
result = model_MLP.wv.most_similar(positive=['france'], topn=20)
print(result)

[('envie', 0.2577926814556122), ('aime', 0.24532631039619446), ('pays', 0.23258820176124573), ('sauver', 0.22628864645957947), ('renvoyer', 0.22321215271949768), ('porter', 0.22119459509849548), ('battre', 0.21434608101844788), ('francais', 0.21272233128547668), ('nations', 0.2058660387992859), ('militaire', 0.2042485922574997), ('bats', 0.20205162465572357), ('changement', 0.19955328106880188), ('destin', 0.19782088696956635), ('gouvernements', 0.1973637342453003), ('mlp_officiel', 0.18885090947151184), ('assurer', 0.18879568576812744), ('decider', 0.18839015066623688), ('civilisation', 0.187577486038208), ('amour', 0.1872176080942154), ('partage', 0.18388047814369202)]


In [52]:
result = model_MLP.wv.most_similar(positive=['etat'], topn=20)
print(result)

[('entreprise', 0.2622726261615753), ('immediatement', 0.2610677182674408), ('prochain', 0.2440115213394165), ('statut', 0.23656286299228668), ('exoneration', 0.23201535642147064), ('mesures', 0.22255879640579224), ('maitrise', 0.21529491245746613), ('considere', 0.21338103711605072), ('souligne', 0.21161247789859772), ('baissera', 0.2070818841457367), ('elysee', 0.20698001980781555), ('epargne', 0.2048940807580948), ('rendre', 0.20442694425582886), ('exceptionnelle', 0.20155829191207886), ('justice', 0.19983521103858948), ('contribuables', 0.19868220388889313), ('baissant', 0.19848966598510742), ('ruralite', 0.1983172744512558), ('presidente', 0.19563473761081696), ('concurrence', 0.19504143297672272)]


In [53]:
result = model_MLP.wv.most_similar(positive=['souverainete'], topn=20)
print(result)

[('alimentaire', 0.49288058280944824), ('frontieres', 0.4397609829902649), ('energetique', 0.3883061110973358), ('industrie', 0.3313864469528198), ('renoncement', 0.32801195979118347), ('crise', 0.3145580291748047), ('mondialisation', 0.3102114200592041), ('desescalade', 0.3057248890399933), ('discuter', 0.2962709963321686), ('independance', 0.2888263165950775), ('assurer', 0.2861247956752777), ('armee', 0.2806982100009918), ('entree', 0.2785166800022125), ('demonstration', 0.27703770995140076), ('sujets', 0.27619561553001404), ('nucleaire', 0.2738054394721985), ('commission', 0.2717369794845581), ('immigration', 0.26906153559684753), ('declaration', 0.2663659453392029), ('constance', 0.261675626039505)]


In [54]:
result = model_MLP.wv.most_similar(positive=['president'], topn=20)
print(result)

[('poissy', 0.352678507566452), ('negocie', 0.3465370833873749), ('jeancastex', 0.3402036130428314), ('presidence', 0.3394870162010193), ('prefet', 0.33615806698799133), ('chequier', 0.3226093649864197), ('premier', 0.3087765574455261), ('laissons', 0.30535826086997986), ('pdt', 0.3039259910583496), ('gerald', 0.29969534277915955), ('lois', 0.2943493723869324), ('exclusif', 0.2884617745876312), ('provocation', 0.28367891907691956), ('menti', 0.2629053592681885), ('corse', 0.2603108286857605), ('compare', 0.2585400640964508), ('zelensky', 0.2553755044937134), ('biden', 0.25531134009361267), ('comporte', 0.2522996962070465), ('ministre', 0.25220444798469543)]


In [55]:
result = model_MLP.wv.most_similar(positive=['politique'], topn=20)
print(result)

[('totalitaire', 0.2763778567314148), ('radical', 0.27081596851348877), ('force', 0.2514961063861847), ('preoccupent', 0.23327012360095978), ('niveau', 0.22616881132125854), ('matiere', 0.22293542325496674), ('fbleunord', 0.2226371318101883), ('constance', 0.21605677902698517), ('catastrophique', 0.21113665401935577), ('droite', 0.20885084569454193), ('preuve', 0.2067064642906189), ('culturelle', 0.2067042887210846), ('cree', 0.20190396904945374), ('traites', 0.2007138878107071), ('puissent', 0.19543296098709106), ('essaye', 0.1946686953306198), ('equidistance', 0.19131416082382202), ('stopper', 0.19110268354415894), ('submersion', 0.19083137810230255), ('stature', 0.19017180800437927)]


In [56]:
result = model_MLP.wv.most_similar(positive=['droit'], topn=20)
print(result)

[('asile', 0.42924100160598755), ('sol', 0.41227245330810547), ('zones', 0.35461506247520447), ('constitution', 0.31295397877693176), ('agira', 0.308268666267395), ('allocations', 0.3034053146839142), ('massacres', 0.29917919635772705), ('fondamental', 0.2909448444843292), ('regroupement', 0.2834262549877167), ('familial', 0.28053921461105347), ('presente', 0.27927306294441223), ('nationalite', 0.2768581211566925), ('corses', 0.2742841839790344), ('loi', 0.2684774696826935), ('referendum', 0.2672514319419861), ('visite', 0.2604047954082489), ('rappeler', 0.25749844312667847), ('etablissements', 0.25313863158226013), ('entendu', 0.2479454129934311), ('familiales', 0.24681958556175232)]


In [57]:
result = model_MLP.wv.most_similar(positive=['entreprise'], topn=20)
print(result)

[('salaries', 0.5468667149543762), ('chefs', 0.4506557881832123), ('exoneration', 0.4437686502933502), ('patronales', 0.43151164054870605), ('salaires', 0.40883609652519226), ('impot', 0.3849179744720459), ('revaloriser', 0.38378840684890747), ('smic', 0.37686261534690857), ('charges', 0.37537211179733276), ('augmenter', 0.37335118651390076), ('etudiants', 0.3649440407752991), ('creer', 0.3614159822463989), ('possibilite', 0.3520083427429199), ('petites', 0.3449292778968811), ('etranger', 0.3370279371738434), ('travaillent', 0.33483052253723145), ('obligation', 0.33354854583740234), ('poursuivre', 0.32735514640808105), ('societes', 0.3231140077114105), ('places', 0.3203541040420532)]


In [58]:
result = model_MLP.wv.most_similar(positive=['economie'], topn=20)
print(result)

[('chomeurs', 0.41166621446609497), ('emplois', 0.35387614369392395), ('dette', 0.33319148421287537), ('brunolemaire', 0.3284732401371002), ('sanctionner', 0.3223462700843811), ('chomage', 0.31960341334342957), ('rester', 0.3034338653087616), ('industriels', 0.29060134291648865), ('reelle', 0.28815361857414246), ('importations', 0.2870390713214874), ('xavierbertrand', 0.28645697236061096), ('payer', 0.2858123779296875), ('entreprises', 0.28525182604789734), ('chiffres', 0.28473106026649475), ('pme', 0.28150317072868347), ('precedent', 0.2794545292854309), ('production', 0.2784939408302307), ('taux', 0.27377012372016907), ('pauvres', 0.27322062849998474), ('beneficier', 0.269746333360672)]


In [59]:
result = model_MLP.wv.most_similar(positive=['emploi'], topn=20)
print(result)

[('chomage', 0.4377748966217041), ('emplois', 0.4236918091773987), ('logements', 0.4046586751937866), ('pme', 0.3666919469833374), ('acces', 0.3547012507915497), ('apprentissage', 0.35216695070266724), ('reduire', 0.3512386083602905), ('taux', 0.34476110339164734), ('activite', 0.3332020044326782), ('jaunes', 0.33302512764930725), ('ras', 0.33161643147468567), ('actuellement', 0.33026203513145447), ('impot', 0.3232487440109253), ('production', 0.3182576894760132), ('bol', 0.31760555505752563), ('exonererai', 0.3117344379425049), ('tpe', 0.3110365569591522), ('vecu', 0.30996373295783997), ('choc', 0.30989471077919006), ('laref', 0.3072410523891449)]


In [60]:
result = model_MLP.wv.most_similar(positive=['travail'], topn=20)
print(result)

[('meilleure', 0.3576391041278839), ('communes', 0.3172711133956909), ('ceux', 0.3118799030780792), ('souhaitent', 0.31172460317611694), ('tot', 0.29881641268730164), ('fassent', 0.29840052127838135), ('entres', 0.2960669994354248), ('reserver', 0.2939426898956299), ('remettre', 0.29287901520729065), ('aides', 0.29284030199050903), ('annuites', 0.2891264259815216), ('travaille', 0.28369274735450745), ('stephane_ravier', 0.2827950716018677), ('apprentissage', 0.2816009819507599), ('patrimoine', 0.27423107624053955), ('confiance', 0.26825299859046936), ('nos', 0.26811420917510986), ('equivalent', 0.26754337549209595), ('implication', 0.2652222216129303), ('engagement', 0.26294928789138794)]


In [61]:
result = model_MLP.wv.most_similar(positive=['taxes'], topn=20)
print(result)

[('decidees', 0.5741530656814575), ('augmentations', 0.5415683388710022), ('tva', 0.5363974571228027), ('hausses', 0.5278366208076477), ('baissant', 0.48395660519599915), ('carburants', 0.48105067014694214), ('centimes', 0.47572246193885803), ('litre', 0.4584499001502991), ('fioul', 0.4510096311569214), ('baissera', 0.4503864347934723), ('baisserai', 0.4322029650211334), ('ticpe', 0.4307597875595093), ('records', 0.42713719606399536), ('prix', 0.38777607679367065), ('augmenter', 0.3871435225009918), ('factures', 0.3851069211959839), ('essence', 0.35922715067863464), ('energies', 0.3545861542224884), ('baisse', 0.3511611223220825), ('purement', 0.3493075668811798)]


In [62]:
result = model_MLP.wv.most_similar(positive=['monde'], topn=20)
print(result)

[('mere', 0.2524833381175995), ('revoir', 0.23963147401809692), ('prepare', 0.2230781763792038), ('bleus', 0.22250013053417206), ('commun', 0.21387739479541779), ('ridicule', 0.21372731029987335), ('entier', 0.20701079070568085), ('lancelot', 0.2068491280078888), ('argument', 0.20676979422569275), ('arrive', 0.20543378591537476), ('poids', 0.20500284433364868), ('fous', 0.20464442670345306), ('sacre', 0.20439565181732178), ('redonner', 0.20294460654258728), ('valeur', 0.20233508944511414), ('bord', 0.19970065355300903), ('particuliere', 0.19890646636486053), ('anti', 0.1976582109928131), ('etrangere', 0.197127565741539), ('journaliste', 0.19235269725322723)]


In [63]:
result = model_MLP.wv.most_similar(positive=['realite'], topn=20)
print(result)

[('terrible', 0.34842416644096375), ('desastre', 0.29437685012817383), ('ramener', 0.28539279103279114), ('faiblesse', 0.2740892767906189), ('bilan', 0.26685401797294617), ('actuel', 0.2650110125541687), ('message', 0.25996774435043335), ('securitaire', 0.25702375173568726), ('montre', 0.24308547377586365), ('consequence', 0.23164090514183044), ('augmentation', 0.22760924696922302), ('ensemble', 0.22263091802597046), ('restrictions', 0.22246307134628296), ('lien', 0.22180016338825226), ('sanitaires', 0.2217053323984146), ('repondre', 0.2196095883846283), ('bfmtv', 0.21959206461906433), ('territoires', 0.21718403697013855), ('avecvous', 0.21645990014076233), ('catastrophique', 0.21630465984344482)]


In [64]:
result = model_MLP.wv.most_similar(positive=['verite'], topn=20)
print(result)

[('tegnererik', 0.5189206004142761), ('macronmckinsey', 0.40395423769950867), ('noir', 0.3929366171360016), ('emotion', 0.3859606981277466), ('victime', 0.35080498456954956), ('sottise', 0.3412255346775055), ('entierement', 0.34096524119377136), ('ecrit', 0.33790120482444763), ('hostile', 0.33254900574684143), ('simple', 0.32547345757484436), ('livre', 0.3164079189300537), ('appele', 0.31526705622673035), ('kiev', 0.3129233717918396), ('reel', 0.3125704526901245), ('argument', 0.3062838315963745), ('preuve', 0.2970302104949951), ('reconquete_z', 0.2950427532196045), ('change', 0.2907830774784088), ('patriotosorus', 0.2807731628417969), ('livrenoirmedia', 0.2799208462238312)]


In [65]:
result = model_MLP.wv.most_similar(positive=['societe'], topn=20)
print(result)

[('langue', 0.4089409410953522), ('inclusive', 0.37186363339424133), ('appliquer', 0.3676133155822754), ('ecriture', 0.3615850508213043), ('racaille', 0.3376776874065399), ('natalite', 0.3303970694541931), ('cet', 0.2657918632030487), ('abri', 0.26464444398880005), ('flux', 0.2645837664604187), ('vise', 0.26144522428512573), ('islamistes', 0.26103734970092773), ('respecter', 0.25810614228248596), ('francaises', 0.2525135278701782), ('harcelement', 0.25100913643836975), ('espace', 0.25088462233543396), ('parle', 0.24726998805999756), ('train', 0.24664992094039917), ('uniforme', 0.245137020945549), ('internationaux', 0.24012981355190277), ('acte', 0.2374456375837326)]


In [66]:
result = model_MLP.wv.most_similar(positive=['pouvoir'], topn=20)
print(result)

[('achat', 0.357227087020874), ('concretes', 0.32177305221557617), ('francais', 0.3209790289402008), ('baissant', 0.3116433918476105), ('solutions', 0.3034991919994354), ('rendra', 0.29000338912010193), ('tva', 0.2877727746963501), ('sociale', 0.28103020787239075), ('insecurite', 0.27652180194854736), ('prosperite', 0.2712661623954773), ('energies', 0.27101948857307434), ('politicienne', 0.26764848828315735), ('preoccupent', 0.2673162519931793), ('jevotemarine', 0.2567182779312134), ('effondrement', 0.25313204526901245), ('cesse', 0.2527087926864624), ('baisse', 0.2509831488132477), ('chomage', 0.2500315308570862), ('carburants', 0.2461475282907486), ('sanitaires', 0.24036720395088196)]


In [67]:
result = model_MLP.wv.most_similar(positive=['avenir'], topn=20)
print(result)

[('destin', 0.3415572941303253), ('langue', 0.32173916697502136), ('fierte', 0.3142007291316986), ('decider', 0.31381139159202576), ('mon', 0.31100600957870483), ('etre', 0.2972348928451538), ('venez', 0.2934952974319458), ('enjeu', 0.29063767194747925), ('civilisation', 0.28508734703063965), ('croit', 0.2732478678226471), ('bats', 0.27119696140289307), ('suivez', 0.2643212378025055), ('souverain', 0.25776174664497375), ('chance', 0.2541224956512451), ('uimm', 0.25279879570007324), ('remporte', 0.24837613105773926), ('ecole', 0.24754071235656738), ('autour', 0.24730099737644196), ('dis', 0.2418070137500763), ('transmission', 0.24152828752994537)]


In [68]:
result = model_MLP.wv.most_similar(positive=['histoire'], topn=20)
print(result)

[('sottise', 0.3865671455860138), ('desescalade', 0.3582479953765869), ('ecrit', 0.33892470598220825), ('fierte', 0.3386448621749878), ('particuliere', 0.3127119839191437), ('moscou', 0.30821505188941956), ('marion', 0.3008155822753906), ('soeur', 0.2898156940937042), ('afpfr', 0.2886526882648468), ('allies', 0.28719571232795715), ('trahi', 0.2823691964149475), ('otan', 0.2819831669330597), ('accord', 0.2786269783973694), ('anti', 0.2769123315811157), ('militaire', 0.2754099369049072), ('oublions', 0.27004411816596985), ('mali', 0.2679131329059601), ('devenue', 0.25675466656684875), ('guerre', 0.25280067324638367), ('gouvernements', 0.25219687819480896)]


In [69]:
result = model_MLP.wv.most_similar(positive=['contre'], topn=20)
print(result)

[('allie', 0.3193697929382324), ('consequences', 0.30394265055656433), ('proteger', 0.26894351840019226), ('fraudes', 0.2679714858531952), ('sanctionner', 0.2569674551486969), ('principales', 0.23758886754512787), ('egard', 0.23534664511680603), ('faut', 0.22676880657672882), ('deserts', 0.22647495567798615), ('victimes', 0.22518782317638397), ('ideologie', 0.22168873250484467), ('prosperite', 0.22117049992084503), ('bras', 0.21870802342891693), ('silence', 0.2057274580001831), ('haute', 0.20385971665382385), ('lutte', 0.20342716574668884), ('penale', 0.20285682380199432), ('monoparentales', 0.201926127076149), ('familles', 0.19875475764274597), ('moyens', 0.1983388066291809)]


In [70]:
result = model_MLP.wv.most_similar(positive=['faut'], topn=20)
print(result)

[('moyens', 0.31509336829185486), ('paix', 0.3003285527229309), ('arreter', 0.29665878415107727), ('veux', 0.29648467898368835), ('voulu', 0.2903573513031006), ('voie', 0.26617759466171265), ('souhaitent', 0.2659722566604614), ('actuelle', 0.2516801655292511), ('reelle', 0.2511676251888275), ('etablissements', 0.2504608929157257), ('difficile', 0.2486356943845749), ('retablir', 0.24400782585144043), ('devons', 0.24276837706565857), ('conditions', 0.23685352504253387), ('beneficier', 0.231195330619812), ('ehpad', 0.23101958632469177), ('prosperite', 0.23064883053302765), ('etranger', 0.2294527143239975), ('contre', 0.22676879167556763), ('propose', 0.2219182550907135)]


In [71]:
result = model_MLP.wv.most_similar(positive=['crises'], topn=20)
print(result)

[('sous', 0.5088121294975281), ('humiliation', 0.3380546271800995), ('general', 0.3326335549354553), ('alliance', 0.3216816782951355), ('verts', 0.31411507725715637), ('profite', 0.3075284957885742), ('donc', 0.3068402409553528), ('subi', 0.3061453104019165), ('responsabilites', 0.3059815764427185), ('suspendu', 0.29566702246665955), ('dynamique', 0.2897944152355194), ('page', 0.2883777320384979), ('objectifgard', 0.2847648561000824), ('desastreux', 0.28424838185310364), ('serie', 0.28389906883239746), ('renoncement', 0.28013452887535095), ('irak', 0.27905869483947754), ('fracture', 0.27498751878738403), ('guadeloupe', 0.2748717665672302), ('acteurs', 0.27469602227211)]
